# Pitch detection by CNN

Of course, there also is a Convolutional Neural Network approach which is trained on data and it better suited for reducing the noise in a signal. This approach is implemented and shown in this notebook.

## References

1. CREPE: A Convolutional Representation for Pitch Estimation (2018)

In [1]:
from IPython.display import display, clear_output, HTML
import crepe
import pyaudio
import os

import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2

from pitch_detection.utilities import *
from pitch_detection.algorithms import *

In [ ]:
# Fix for Tensorflow on MacOS
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

global keep_going
keep_going = True

audio = pyaudio.PyAudio()

# The algorithm was trained on a sample rate of 16kHz, so to overcome resampling, 16kHz is used for recording
frames_per_buffer = 256
frames_per_second = 16000
stream = audio.open(format=pyaudio.paInt16,
                    channels=1,
                    frames_per_buffer=frames_per_buffer,
                    rate=frames_per_second,
                    input=True)

xs = []
def process_audio(chunk_bytes):
    """Small helper function for converting a stream of bytes to the fundamental frequency and visualizing it."""
    chunk_data = np.frombuffer(chunk_bytes, np.int16)
    time, frequency, confidence, activation = crepe.predict(chunk_data, frames_per_second, viterbi=True, verbose=False, model_capacity='tiny')
    index = np.argmax(confidence)
    if np.max(confidence) > 0.2:
        freq = frequency[index]
        tone = convert_freq_to_tone(freq) + " (" + str(int(freq)) + " Hz)"
        html = '<span style="background-color: rgb(0, {:d}, 0); padding: 20px 0; width: 100px; text-align: center; display: inline-block;">{:s}</span>'.format(int(155 + 100 * np.max(confidence)), tone)
        clear_output(wait=True)
        display(HTML(html))

process_audio(stream.read(frames_per_buffer, exception_on_overflow=False))
while keep_going:
    try:
        process_audio(stream.read(frames_per_buffer, exception_on_overflow=False))
    except KeyboardInterrupt:
        keep_going = False
    except Exception as e:
        raise e
    
stream.stop_stream()
stream.close()

audio.terminate()